In [ ]:
import pandas as pd
GROUND_TRUTH_METADATA_FILE = "/home/etaylor/code_projects/thesis/metadata/ground_truth_metadata.csv"

### Option 1 - Load the ground truth dataset

In [ ]:
# load the ground_truth_df from the csv file
ground_truth_df = pd.read_csv(GROUND_TRUTH_METADATA_FILE)
ground_truth_df.head()

### Option 2 - Calculate the Ground Truth Dataset

In [ ]:
import pickle
import pandas as pd
import config

#load annotated data
with open('/home/etaylor/code_projects/thesis/metadata/images_annotated.pkl', 'rb') as f:
    users_annotated_data = pickle.load(f)
    
users_annotated_df = pd.DataFrame(users_annotated_data, columns=['image_number'])
users_annotated_df.head()

,image_number
0,IMG_2129
1,IMG_2198
2,IMG_0546
3,IMG_1079
4,IMG_0543


In [ ]:

# Get the ground truth from the annotated metadata csv
annotation_tracking_df = pd.read_csv(config.ANNOTATIONS_TRACKING_METADATA_FILE)

# filter all records where done column is True
annotation_tracking_df = annotation_tracking_df[annotation_tracking_df.done == True]
print(f"Number of records in the annotation tracking metadata file: {len(annotation_tracking_df)}")

# convert Time column to datetime from strtime
annotation_tracking_df['Time'] = pd.to_datetime(annotation_tracking_df['Time'], format='%d-%m-%Y_%H-%M-%S')

# filter all records that the image_number are in users_annotated_df or the Time is past 2024
annotation_working_df = annotation_tracking_df[annotation_tracking_df.image_number.isin(users_annotated_df.image_number) | (annotation_tracking_df.Time > '2024-01-01')]
print(f"Number of records in the annotation tracking metadata file that are also annotated by users: {len(annotation_working_df)}")
annotation_working_df.head()

Number of records in the annotation tracking metadata file: 55
Number of records in the annotation tracking metadata file that are also annotated by users: 50


,image_number,annotator,Time,done
0,IMG_2198,dorins,2023-12-15 19:26:10,True
1,IMG_2153,dorins,2023-12-15 19:26:10,True
3,IMG_2145,Nirmalka,2023-12-15 19:26:10,True
4,IMG_2129,Nirmalka,2023-12-15 19:26:10,True
5,IMG_1875,dorins,2023-12-15 19:26:10,True


In [ ]:
from src.annotation_handling.segmentsai_handler import SegmentsAIHandler
segments_handler = SegmentsAIHandler()

ground_truth_trichome_distribution = {}
# get the trichome distribution for each annotated image dataset
for image_number in annotation_working_df.image_number:
    print(f"get the distribution for image {image_number}")
    trichome_distribution = segments_handler.get_trichome_distribution(image_number)
    ground_truth_trichome_distribution[image_number] = trichome_distribution
    print(f"Distribution for image {image_number}: \n{trichome_distribution}")

INFO:segments.client:Initialized successfully.


get the distribution for image IMG_2198
Distribution for image IMG_2198: 
{'clear': 48, 'cloudy': 148, 'amber': 10}
get the distribution for image IMG_2153
Distribution for image IMG_2153: 
{'clear': 38, 'cloudy': 120, 'amber': 28}
get the distribution for image IMG_2145
Distribution for image IMG_2145: 
{'clear': 51, 'cloudy': 110, 'amber': 61}
get the distribution for image IMG_2129
Distribution for image IMG_2129: 
{'clear': 75, 'cloudy': 216, 'amber': 9}
get the distribution for image IMG_1875
Distribution for image IMG_1875: 
{'clear': 7, 'cloudy': 77, 'amber': 9}
get the distribution for image IMG_1787
Distribution for image IMG_1787: 
{'clear': 3, 'cloudy': 104, 'amber': 60}
get the distribution for image IMG_1857
Distribution for image IMG_1857: 
{'clear': 7, 'cloudy': 163, 'amber': 17}
get the distribution for image IMG_1753
Distribution for image IMG_1753: 
{'clear': 74, 'cloudy': 21, 'amber': 0}
get the distribution for image IMG_1818
Distribution for image IMG_1818: 
{'clea

In [ ]:
# organize the ground truth to a df
images_annotations = []
# create a csv file for each image path and the trichome value
for image_number, image_dist in ground_truth_trichome_distribution.items():
    image_path = config.get_image_path(image_number)
    week, zoom_type = config.find_image_details(image_number)
    images_annotations.append({"image_number": image_number,
                                "image_path":image_path,
                                "week_number": week,
                                "clear": image_dist.get("clear", 0),
                                "cloudy": image_dist.get("cloudy", 0),
                                "amber": image_dist.get("amber", 0)})
ground_truth_df = pd.DataFrame(images_annotations)

# save the ground_truth_df to a csv file with timestamp
ground_truth_df.to_csv(GROUND_TRUTH_METADATA_FILE, index=False)

ground_truth_df.head()

,image_number,image_path,week_number,clear,cloudy,amber
0,IMG_2198,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,48,148,10
1,IMG_2153,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,38,120,28
2,IMG_2145,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,51,110,61
3,IMG_2129,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,75,216,9
4,IMG_1875,/sise/home/etaylor/images/raw_images/week8_07_...,week8_07_06_2023,7,77,9


In [ ]:
ground_truth_df['week_number'].value_counts()

week_number
week9_15_06_2023    13
week6_22_05_2023    11
week5_18_05_2023    10
week7_01_06_2023     9
week8_07_06_2023     7
Name: count, dtype: int64

### Preprocess patches of the train and test data for infrence with the models

In [4]:
from src.pipelines.preprocessing_pipeline import preprocess_single_image


# preprocess each image in the df to a dict
preprocessed_images = {}

for index, row in ground_truth_df.iterrows():
    image_number = row["image_number"]
    image_path = row["image_path"]
    preprocessed_image = preprocess_single_image(image_path, image_number)
    preprocessed_images[image_number] = preprocessed_image


### Evaluation Function

Here we should add all the eval functions.
We should use both classification and regression metrics.
For the classification we will use precision, recall, mAP, Confusion Matrix
We should use RMSE,MSE, MAE, R-squared

In [5]:
import numpy as np

def calculate_rmse(ground_truth_df, preds_df):
    # Merge the DataFrames on the image_number column
    merged_df = pd.merge(ground_truth_df, preds_df, on="image_number", suffixes=('_actual', '_pred'))

    # Calculate squared differences for each trichome type
    merged_df['clear_diff'] = (merged_df['clear_actual'] - merged_df['clear_pred']) ** 2
    merged_df['cloudy_diff'] = (merged_df['cloudy_actual'] - merged_df['cloudy_pred']) ** 2
    merged_df['amber_diff'] = (merged_df['amber_actual'] - merged_df['amber_pred']) ** 2

    # Calculate RMSE for each trichome type
    rmse_clear = np.sqrt(merged_df['clear_diff'].mean())
    rmse_cloudy = np.sqrt(merged_df['cloudy_diff'].mean())
    rmse_amber = np.sqrt(merged_df['amber_diff'].mean())

    # Calculate overall RMSE
    merged_df['total_diff'] = merged_df[['clear_diff', 'cloudy_diff', 'amber_diff']].sum(axis=1)
    overall_rmse = np.sqrt(merged_df['total_diff'].mean())

    # Return a dictionary containing RMSE for each class and the overall RMSE
    return {
        "RMSE Clear": rmse_clear,
        "RMSE Cloudy": rmse_cloudy,
        "RMSE Amber": rmse_amber,
        "Overall RMSE": overall_rmse
    }

### Inference Using Detectron2 Mask RCNN

In [6]:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

# load the best model config and weights
checkpoint_path = "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/model_final.pth"
config_yaml_path = "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/config.yaml"
cfg = get_cfg()
# load config
cfg.merge_from_file(config_yaml_path)
# define predictor
# load checkpoint
cfg.MODEL.WEIGHTS = checkpoint_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

[04/19 19:32:50 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/model_final.pth ...


INFO:fvcore.common.checkpoint:[Checkpointer] Loading from /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/model_final.pth ...


In [10]:
def detect_trichome_dist_with_detectron(image_number, preprocessed_image, predictor):
    
    class_counts = {'clear': 0, 'cloudy': 0, 'amber': 0}
    
    # get the prediction
    for image, _ in preprocessed_image[image_number]:
        # predict the image
        outputs = predictor(image)
        
        # Extract predictions
        pred_classes = outputs['instances'].pred_classes.cpu().tolist()
        
        for class_id in pred_classes:
            class_counts[config.ANNOTATIONS_CLASS_MAPPINGS[class_id]] += 1
            
    return class_counts

In [21]:
# create predictions pandas dataframe using detectron2
detectron_preds = []
for image_number in preprocessed_images:
    print(f"Predicting image {image_number}")
    trichome_dist = detect_trichome_dist_with_detectron(image_number, preprocessed_images, predictor)
    detectron_preds.append({"image_number": image_number,
                            "clear": trichome_dist["clear"],
                            "cloudy": trichome_dist["cloudy"],
                            "amber": trichome_dist["amber"]})

detectron_preds_df = pd.DataFrame(detectron_preds)
detectron_preds_df.head()

Predicting image IMG_2129
Predicting image IMG_2198
Predicting image IMG_0546
Predicting image IMG_1079
Predicting image IMG_0543
Predicting image IMG_1818
Predicting image IMG_0540
Predicting image IMG_2153
Predicting image IMG_0016
Predicting image IMG_0542
Predicting image IMG_1085
Predicting image IMG_0014
Predicting image IMG_1875
Predicting image IMG_1857
Predicting image IMG_0545
Predicting image IMG_2285
Predicting image IMG_2242
Predicting image IMG_1787
Predicting image IMG_1082
Predicting image IMG_0017
Predicting image IMG_1753
Predicting image IMG_0003
Predicting image IMG_2145
Predicting image IMG_1093
Predicting image IMG_0547
Predicting image IMG_0001


,image_number,clear,cloudy,amber
0,IMG_2129,86,245,16
1,IMG_2198,48,175,11
2,IMG_0546,69,76,3
3,IMG_1079,88,109,5
4,IMG_0543,98,188,4


In [23]:
rmse_results = calculate_rmse(ground_truth_df, detectron_preds_df)
print(rmse_results)

{'RMSE Clear': 29.454005865730075, 'RMSE Cloudy': 26.70061941085032, 'RMSE Amber': 8.833938053804856, 'Overall RMSE': 40.72468538859447}


### Inference Using RT DETR

In [6]:
from ultralytics import RTDETR

model_checkpoint_path = '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/RT_DETR_train9/weights/best.pt'
model = RTDETR(model_checkpoint_path)

In [7]:
results = model(preprocessed_images['IMG_0001'][5][0], imgsz=512)


0: 512x512 8 clears, 44 cloudys, 1 amber, 76.6ms
Speed: 2.0ms preprocess, 76.6ms inference, 15.5ms postprocess per image at shape (1, 3, 512, 512)


In [8]:
def detect_trichome_dist_with_ultralytics(image_number, preprocessed_image, predictor):
    
    class_counts = {'clear': 0, 'cloudy': 0, 'amber': 0}
    
    ultralytics_mapping_dict = {0: 'clear', 1: 'cloudy', 2: 'amber'}
    
    # get the prediction
    for image, _ in preprocessed_image[image_number]:
        # predict the image
        outputs = predictor(image, imgsz=512, verbose=False)
        
        for pred in outputs:
            for c in pred.boxes.cls:
                class_counts[ultralytics_mapping_dict[int(c)]] += 1
            
    return class_counts

In [19]:
# create predictions pandas dataframe using detectron2
RT_DETR_preds = []
for image_number in preprocessed_images:
    print(f"Predicting image {image_number}")
    trichome_dist = detect_trichome_dist_with_ultralytics(image_number, preprocessed_images, model)
    RT_DETR_preds.append({"image_number": image_number,
                            "clear": trichome_dist["clear"],
                            "cloudy": trichome_dist["cloudy"],
                            "amber": trichome_dist["amber"]})

RT_DETR_preds_df = pd.DataFrame(RT_DETR_preds)
RT_DETR_preds_df.head()

Predicting image IMG_2129


Predicting image IMG_2198
Predicting image IMG_0546
Predicting image IMG_1079
Predicting image IMG_0543
Predicting image IMG_1818
Predicting image IMG_0540
Predicting image IMG_2153
Predicting image IMG_0016
Predicting image IMG_0542
Predicting image IMG_1085
Predicting image IMG_0014
Predicting image IMG_1875
Predicting image IMG_1857
Predicting image IMG_0545
Predicting image IMG_2285
Predicting image IMG_2242
Predicting image IMG_1787
Predicting image IMG_1082
Predicting image IMG_0017
Predicting image IMG_1753
Predicting image IMG_0003
Predicting image IMG_2145
Predicting image IMG_1093
Predicting image IMG_0547
Predicting image IMG_0001


,image_number,clear,cloudy,amber
0,IMG_2129,56,254,43
1,IMG_2198,26,243,30
2,IMG_0546,29,110,35
3,IMG_1079,60,223,13
4,IMG_0543,51,232,31


In [20]:
rmse_results = calculate_rmse(ground_truth_df, RT_DETR_preds_df)
print(rmse_results)

{'RMSE Clear': 23.456178454563048, 'RMSE Cloudy': 82.99397568498571, 'RMSE Amber': 23.17657969058088, 'Overall RMSE': 89.30479356588958}


### Inference Using Yolo V5

In [11]:
from ultralytics import YOLO

model_yolov5_checkpoint_path = '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/YOLOV5_train8/weights/best.pt'
model_yolov5 = YOLO(model_yolov5_checkpoint_path)

In [12]:
# create predictions pandas dataframe using detectron2
yolov5_preds = []
for image_number in preprocessed_images:
    print(f"Predicting image {image_number}")
    trichome_dist = detect_trichome_dist_with_ultralytics(image_number, preprocessed_images, model_yolov5)
    yolov5_preds.append({"image_number": image_number,
                            "clear": trichome_dist["clear"],
                            "cloudy": trichome_dist["cloudy"],
                            "amber": trichome_dist["amber"]})

yolov5_preds_df = pd.DataFrame(yolov5_preds)
yolov5_preds_df.head()

Predicting image IMG_2129
Predicting image IMG_2198
Predicting image IMG_0546
Predicting image IMG_1079
Predicting image IMG_0543
Predicting image IMG_1818
Predicting image IMG_0540
Predicting image IMG_2153
Predicting image IMG_0016
Predicting image IMG_0542
Predicting image IMG_1085
Predicting image IMG_0014
Predicting image IMG_1875
Predicting image IMG_1857
Predicting image IMG_0545
Predicting image IMG_2285
Predicting image IMG_2242
Predicting image IMG_1787
Predicting image IMG_1082
Predicting image IMG_0017
Predicting image IMG_1753
Predicting image IMG_0003
Predicting image IMG_2145
Predicting image IMG_1093
Predicting image IMG_0547
Predicting image IMG_0001


,image_number,clear,cloudy,amber
0,IMG_2129,112,283,17
1,IMG_2198,65,216,15
2,IMG_0546,58,115,11
3,IMG_1079,101,172,8
4,IMG_0543,90,259,7


In [13]:
rmse_results = calculate_rmse(ground_truth_df, yolov5_preds_df)
print(rmse_results)

{'RMSE Clear': 33.27681105283609, 'RMSE Cloudy': 70.24380618739497, 'RMSE Amber': 11.924441609245596, 'Overall RMSE': 78.63670115938721}


### Inference Using Yolo V8

In [21]:
from ultralytics import YOLO

model_yolov8_checkpoint_path = '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/YOLOV8_train2/weights/best.pt'
model_yolov8 = YOLO(model_yolov8_checkpoint_path)

In [22]:
# create predictions pandas dataframe using detectron2
yolov8_preds = []
for image_number in preprocessed_images:
    print(f"Predicting image {image_number}")
    trichome_dist = detect_trichome_dist_with_ultralytics(image_number, preprocessed_images, model_yolov8)
    yolov8_preds.append({"image_number": image_number,
                            "clear": trichome_dist["clear"],
                            "cloudy": trichome_dist["cloudy"],
                            "amber": trichome_dist["amber"]})

yolov8_preds_df = pd.DataFrame(yolov8_preds)
yolov8_preds_df.head()

Predicting image IMG_2129
Predicting image IMG_2198
Predicting image IMG_0546
Predicting image IMG_1079
Predicting image IMG_0543
Predicting image IMG_1818
Predicting image IMG_0540
Predicting image IMG_2153
Predicting image IMG_0016
Predicting image IMG_0542
Predicting image IMG_1085
Predicting image IMG_0014
Predicting image IMG_1875
Predicting image IMG_1857
Predicting image IMG_0545
Predicting image IMG_2285
Predicting image IMG_2242
Predicting image IMG_1787
Predicting image IMG_1082
Predicting image IMG_0017
Predicting image IMG_1753
Predicting image IMG_0003
Predicting image IMG_2145
Predicting image IMG_1093
Predicting image IMG_0547
Predicting image IMG_0001


,image_number,clear,cloudy,amber
0,IMG_2129,156,233,15
1,IMG_2198,51,247,10
2,IMG_0546,53,120,4
3,IMG_1079,72,154,5
4,IMG_0543,82,228,3


In [23]:
rmse_results = calculate_rmse(ground_truth_df, yolov8_preds_df)
print(rmse_results)

{'RMSE Clear': 45.738806953597525, 'RMSE Cloudy': 44.85446551819648, 'RMSE Amber': 11.283071049483484, 'Overall RMSE': 65.04820697582086}


### Inference Using Yolo V9

In [6]:
from ultralytics import YOLO

model_yolov9_checkpoint_path = '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/YOLOV9_train7/weights/best.pt'
model_yolov9 = YOLO(model_yolov9_checkpoint_path)

In [9]:
# create predictions pandas dataframe using detectron2
yolov9_preds = []
for image_number in preprocessed_images:
    print(f"Predicting image {image_number}")
    trichome_dist = detect_trichome_dist_with_ultralytics(image_number, preprocessed_images, model_yolov9)
    yolov9_preds.append({"image_number": image_number,
                            "clear": trichome_dist["clear"],
                            "cloudy": trichome_dist["cloudy"],
                            "amber": trichome_dist["amber"]})

yolov9_preds_df = pd.DataFrame(yolov9_preds)
yolov9_preds_df.head()

Predicting image IMG_2129
Predicting image IMG_2198
Predicting image IMG_0546
Predicting image IMG_1079
Predicting image IMG_0543
Predicting image IMG_1818
Predicting image IMG_0540
Predicting image IMG_2153
Predicting image IMG_0016
Predicting image IMG_0542
Predicting image IMG_1085
Predicting image IMG_0014
Predicting image IMG_1875
Predicting image IMG_1857
Predicting image IMG_0545
Predicting image IMG_2285
Predicting image IMG_2242
Predicting image IMG_1787
Predicting image IMG_1082
Predicting image IMG_0017
Predicting image IMG_1753
Predicting image IMG_0003
Predicting image IMG_2145
Predicting image IMG_1093
Predicting image IMG_0547
Predicting image IMG_0001


,image_number,clear,cloudy,amber
0,IMG_2129,59,218,47
1,IMG_2198,22,246,18
2,IMG_0546,44,111,22
3,IMG_1079,73,198,13
4,IMG_0543,40,196,26


In [10]:
rmse_results = calculate_rmse(ground_truth_df, yolov9_preds_df)
print(rmse_results)

{'RMSE Clear': 28.55830742668275, 'RMSE Cloudy': 67.02812268569984, 'RMSE Amber': 26.54893884595303, 'Overall RMSE': 77.54477614702559}


### Compare Models RMSE Scores

In [16]:
mask_rcnn_score = {'RMSE Clear': 29.454005865730075, 'RMSE Cloudy': 26.70061941085032, 'RMSE Amber': 8.833938053804856, 'Overall RMSE': 40.72468538859447}
rt_detr_score = {'RMSE Clear': 23.456178454563048, 'RMSE Cloudy': 82.99397568498571, 'RMSE Amber': 23.17657969058088, 'Overall RMSE': 89.30479356588958}
yolov5_score = {'RMSE Clear': 33.27681105283609, 'RMSE Cloudy': 70.24380618739497, 'RMSE Amber': 11.924441609245596, 'Overall RMSE': 78.63670115938721}
yolov8_score = {'RMSE Clear': 45.738806953597525, 'RMSE Cloudy': 44.85446551819648, 'RMSE Amber': 11.283071049483484, 'Overall RMSE': 65.04820697582086}
yolov9_score = {'RMSE Clear': 28.55830742668275, 'RMSE Cloudy': 67.02812268569984, 'RMSE Amber': 26.54893884595303, 'Overall RMSE': 77.54477614702559}

# create scores df
scores_df = pd.DataFrame([mask_rcnn_score, rt_detr_score, yolov5_score, yolov8_score, yolov9_score], index=["Mask RCNN", "RT-DETR", "YOLOv5", "YOLOv8", "YOLOv9"])

In [17]:
scores_df

,RMSE Clear,RMSE Cloudy,RMSE Amber,Overall RMSE
Mask RCNN,29.454006,26.700619,8.833938,40.724685
RT-DETR,23.456178,82.993976,23.176580,89.304794
YOLOv5,33.276811,70.243806,11.924442,78.636701
YOLOv8,45.738807,44.854466,11.283071,65.048207
YOLOv9,28.558307,67.028123,26.548939,77.544776
